In [1]:
)clear
⎕IO ← 0
⎕CT ← 1e¯10
Assert ← {⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←0}   ⍝ by Roger Hui
Table  ← {⎕PP←3 ⋄ ⍕¨¨⍺,[¯0.5]⍵}                              ⍝ display as table

clear ws

# Matrix divide
The [`⌹` function](https://help.dyalog.com/latest/Content/Language/Primitive%20Functions/Matrix%20Divide.htm), when used dyadically (`b⌹m`), performs the matrix division of `b` by `m`. In case `b` is a vector and `m` is a matrix with shape `2⍴≢b`, the result of `b⌹m` will be the vector `x` which solves the linear system `b ≡ m +.× x`.

In [2]:
b m←{?⍵⍴0}¨3 (3 3)  ⍝ random sample data
'bmx' Table b m (x←b⌹m)
Assert b ≡ m +.× x

┌─────────────────┬────────────────────┬───────────────────┐
│b │m │x │
├─────────────────┼────────────────────┼───────────────────┤
│┌─────┬─────┬───┐│┌─────┬──────┬─────┐│┌─────┬─────┬─────┐│
││0.508│0.457│0.5│││0.205│0.506 │0.35 │││0.496│0.459│0.496││
│└─────┴─────┴───┘│├─────┼──────┼─────┤│└─────┴─────┴─────┘│
│ ││0.773│0.0265│0.123││ │
│ │├─────┼──────┼─────┤│ │
│ ││0.506│0.417 │0.116││ │
│ │└─────┴──────┴─────┘│ │
└─────────────────┴────────────────────┴───────────────────┘

# Matrix divide of nested vector elements

Sometimes, we do not need to solve a linear system, but many linear systems. What if every element of our vector and every element of our matrix was an array with several values? Let's try it. We are going to divide a vector of 2-elements vectors by a matrix of 2-elements vectors:

In [3]:
m b←2{↓?(⍵,⍺)⍴0}¨(3 3) 3  ⍝ random sample data
(,⍉'' '⍴' '⍴¨'∘.,'bm') Table b (⍴b) (⍴¨b) m (⍴m) (⍴¨m)
b⌹m

┌──────────────────────────────────────┬───┬───────┬────────────────────────────────────────┬─────┬───────┐
│b │⍴b │⍴¨b │m │⍴m │⍴¨m │
├──────────────────────────────────────┼───┼───────┼────────────────────────────────────────┼─────┼───────┤
│┌───────────┬───────────┬────────────┐│┌─┐│┌─┬─┬─┐│┌────────────┬────────────┬────────────┐│┌─┬─┐│┌─┬─┬─┐│
││0.564 0.052│0.203 0.864│0.0701 0.542│││3│││2│2│2│││0.0715 0.694│0.918 0.074 │0.559 0.897 │││3│3│││2│2│2││
│└───────────┴───────────┴────────────┘│└─┘│└─┴─┴─┘│├────────────┼────────────┼────────────┤│└─┴─┘│├─┼─┼─┤│
│ │ │ ││0.196 0.939 │0.574 0.966 │0.0354 0.423││ ││2│2│2││
│ │ │ │├────────────┼────────────┼────────────┤│ │├─┼─┼─┤│
│ │ │ ││0.397 0.741 │0.303 0.0568│0.495 0.863 ││ ││2│2│2││
│ │ │ │└────────────┴────────────┴────────────┘│ │└─┴─┴─┘│
└──────────────────────────────────────┴───┴───────┴────────────────────────────────────────┴─────┴───────┘

DOMAIN ERROR
      b⌹m
       ∧


It does not work. We need to restructure the data so that we can use the [rank operator](https://help.dyalog.com/latest/Content/Language/Primitive%20Operators/Rank.htm), `⍤`. First, we mix the arrays, to get arrays of more dimensions:

In [4]:
(,'' '⍴'∘.,(⊂'↑[0]'),¨'bm') Table (⊢,⍴¨)↑[0]¨b m

┌────────────────────┬──────────────────────┬──────┬───────┐
│↑[0]b │↑[0]m │⍴↑[0]b│⍴↑[0]m │
├────────────────────┼──────────────────────┼──────┼───────┤
│┌─────┬─────┬──────┐│┌──────┬──────┬──────┐│┌─┬─┐ │┌─┬─┬─┐│
││0.564│0.203│0.0701│││0.0715│0.918 │0.559 │││2│3│ ││2│3│3││
│├─────┼─────┼──────┤│├──────┼──────┼──────┤│└─┴─┘ │└─┴─┴─┘│
││0.052│0.864│0.542 │││0.196 │0.574 │0.0354││ │ │
│└─────┴─────┴──────┘│├──────┼──────┼──────┤│ │ │
│ ││0.397 │0.303 │0.495 ││ │ │
│ │└──────┴──────┴──────┘│ │ │
│ │┌──────┬──────┬──────┐│ │ │
│ ││0.694 │0.074 │0.897 ││ │ │
│ │├──────┼──────┼──────┤│ │ │
│ ││0.939 │0.966 │0.423 ││ │ │
│ │├──────┼──────┼──────┤│ │ │
│ ││0.741 │0.0568│0.863 ││ │ │
│ │└──────┴──────┴──────┘│ │ │
└────────────────────┴──────────────────────┴──────┴───────┘

Now, matrix divide with the rank operator will work:

In [5]:
'bmx' Table b m (x←b⌹⍤1 2⍥(↑[0])m)

┌──────────────────────────────────────┬────────────────────────────────────────┬────────────────────┐
│b │m │x │
├──────────────────────────────────────┼────────────────────────────────────────┼────────────────────┤
│┌───────────┬───────────┬────────────┐│┌────────────┬────────────┬────────────┐│┌──────┬─────┬─────┐│
││0.564 0.052│0.203 0.864│0.0701 0.542│││0.0715 0.694│0.918 0.074 │0.559 0.897 │││¯0.505│0.512│0.233││
│└───────────┴───────────┴────────────┘│├────────────┼────────────┼────────────┤│├──────┼─────┼─────┤│
│ ││0.196 0.939 │0.574 0.966 │0.0354 0.423│││6.13 │¯3.12│¯4.43││
│ │├────────────┼────────────┼────────────┤│└──────┴─────┴─────┘│
│ ││0.397 0.741 │0.303 0.0568│0.495 0.863 ││ │
│ │└────────────┴────────────┴────────────┘│ │
└──────────────────────────────────────┴────────────────────────────────────────┴────────────────────┘

We already solved the linear system. However, the solution does not have the right structure:

In [6]:
('' '⍴' '⍴¨',¨'x') Table x (⍴x) (⍴¨x)
b ≡ m +.× x

┌────────────────────┬─────┬────┐
│x │⍴x │⍴¨x │
├────────────────────┼─────┼────┤
│┌──────┬─────┬─────┐│┌─┬─┐│┌┬┬┐│
││¯0.505│0.512│0.233│││2│3│││││││
│├──────┼─────┼─────┤│└─┴─┘│├┼┼┤│
││6.13 │¯3.12│¯4.43││ ││││││
│└──────┴─────┴─────┘│ │└┴┴┘│
└────────────────────┴─────┴────┘

LENGTH ERROR
      b≡m+.×x
          ∧


We need to restructure the array again, so we split it, removing the first axis that we added before. This is getting a bit complicated, so we will define a *nested divide* function:

In [7]:
⎕ ← ND ← ↓[0]⌹⍤1 2⍥(↑[0])  ⍝ nested matrix divide
('' '⍴' '⍴¨',¨'x') Table x (⍴x) (⍴¨x←b ND m)
Assert b ≡ m +.× x

┌─┴──┐
 [0] ⍥
┌─┘ ┌──┴───┐
↓ ⍤ [0]
 ┌┴┐ ┌─┘
 ⌹ 1 2 ↑

┌─────────────────────────────────────┬───┬───────┐
│x │⍴x │⍴¨x │
├─────────────────────────────────────┼───┼───────┤
│┌───────────┬───────────┬───────────┐│┌─┐│┌─┬─┬─┐│
││¯0.505 6.13│0.512 ¯3.12│0.233 ¯4.43│││3│││2│2│2││
│└───────────┴───────────┴───────────┘│└─┘│└─┴─┴─┘│
└─────────────────────────────────────┴───┴───────┘

We solved the two linear systems now! This time, we got a solution array `x` with exactly the same structure as `b`, so there is no error.

# Matrix divide of nested array elements

We are not done yet. What if our nested elements are not simple vectors but arrays with rank higher than 1?

In [8]:
RN ← {5×⊂⍤(≢⍺)?(⍵,⍺)⍴0}  ⍝ random nested. ⍺ shape of element, ⍵ shape of array
m b←(⊂3 4)RN¨(2 2) 2  ⍝ random sample data
('' '⍴' '⍴¨',¨'m') Table m (⍴m) (⍴¨m)
('' '⍴' '⍴¨',¨'b') Table b (⍴b) (⍴¨b)
('' '⍴' '⍴¨',¨'x') Table x (⍴x) (⍴¨x←b ND m)
b ≡ m +.× x

┌─────────────────────────────────────────────┬─────┬─────────┐
│m │⍴m │⍴¨m │
├─────────────────────────────────────────────┼─────┼─────────┤
│┌─────────────────────┬─────────────────────┐│┌─┬─┐│┌───┬───┐│
││2.88 2.79 4.01 2.81 │2.97 1.99 1.58 1.58 │││2│2│││3 4│3 4││
││4.84 4.54 0.814 4.2 │4.82 1.14 2.69 1.36 ││└─┴─┘│├───┼───┤│
││2.92 1.32 4.79 3.56 │1.04 1.09 1.18 3.38 ││ ││3 4│3 4││
│├─────────────────────┼─────────────────────┤│ │└───┴───┘│
││1.54 0.261 1.25 0.642│3.3 3.18 3.81 2.06││ │ │
││1.02 4.87 2.19 0.712│0.186 2.96 1.98 4.39││ │ │
││1.15 3.16 1.58 0.624│4.14 0.606 2.76 4.83││ │ │
│└─────────────────────┴─────────────────────┘│ │ │
└─────────────────────────────────────────────┴─────┴─────────┘

┌──────────────────────────────────────────────┬───┬─────────┐
│b │⍴b │⍴¨b │
├──────────────────────────────────────────────┼───┼─────────┤
│┌────────────────────┬───────────────────────┐│┌─┐│┌───┬───┐│
││2.45 2.75 3.92 3.51│1.05 2 2.21 2.58 │││2│││3 4│3 4││
││0.404 3.29 3.69 1.61│0.312 2.77 2.36 0.527 ││└─┘│└───┴───┘│
││2.89 4.73 1.58 1.21│4.14 3.78 1.74 0.00454││ │ │
│└────────────────────┴───────────────────────┘│ │ │
└──────────────────────────────────────────────┴───┴─────────┘

┌────────────────────────────────────────┬─────┬─────┐
│x │⍴x │⍴¨x │
├────────────────────────────────────────┼─────┼─────┤
│┌──────────────────┬───────────────────┐│┌─┬─┐│┌─┬─┐│
││1 0.354 0.702 │¯0.152 ¯0.272 0.805│││4│2│││3│3││
│├──────────────────┼───────────────────┤│└─┴─┘│├─┼─┤│
││0.57 0.835 0.472 │0.582 ¯0.436 3.78 ││ ││3│3││
│├──────────────────┼───────────────────┤│ │├─┼─┤│
││0.859 ¯0.224 0.203│0.297 1.44 0.515 ││ ││3│3││
│├──────────────────┼───────────────────┤│ │├─┼─┤│
││0.66 0.364 0.385 │1.04 0.061 ¯0.0488 ││ ││3│3││
│└──────────────────┴───────────────────┘│ │└─┴─┘│
└────────────────────────────────────────┴─────┴─────┘

LENGTH ERROR
      b≡m+.×x
          ∧


We actually solved the system, but obviously `↓[0]` was not enough to restructure the result value as needed. We only removed one axis, but we added two with `↑[0]`. Since we want to nest more than one axis, we will use the [enclose function](), `⊂`, together with the rank operator, instead of split. But there is another problem. Let's have a look at the shape of the result of the matrix divide:

In [9]:
('' '⍴' '⍴¨',¨'d') Table d (⍴d) (⍴¨d←b⌹⍤1 2⍥(↑[0])m) 

┌────────────────┬───────┬───┐
│d │⍴d │⍴¨d│
├────────────────┼───────┼───┤
│┌──────┬───────┐│┌─┬─┬─┐│┌┬┐│
││1 │¯0.152 │││3│4│2││││││
│├──────┼───────┤│└─┴─┴─┘│├┼┤│
││0.57 │0.582 ││ │││││
│├──────┼───────┤│ │├┼┤│
││0.859 │0.297 ││ │││││
│├──────┼───────┤│ │├┼┤│
││0.66 │1.04 ││ │││││
│└──────┴───────┘│ │└┴┘│
│┌──────┬───────┐│ │┌┬┐│
││0.354 │¯0.272 ││ │││││
│├──────┼───────┤│ │├┼┤│
││0.835 │¯0.436 ││ │││││
│├──────┼───────┤│ │├┼┤│
││¯0.224│1.44 ││ │││││
│├──────┼───────┤│ │├┼┤│
││0.364 │0.061 ││ │││││
│└──────┴───────┘│ │└┴┘│
│┌──────┬───────┐│ │┌┬┐│
││0.702 │0.805 ││ │││││
│├──────┼───────┤│ │├┼┤│
││0.472 │3.78 ││ │││││
│├──────┼───────┤│ │├┼┤│
││0.203 │0.515 ││ │││││
│├──────┼───────┤│ │├┼┤│
││0.385 │¯0.0488││ │││││
│└──────┴───────┘│ │└┴┘│
└────────────────┴───────┴───┘

What we want to do is to have the same structure of `b` (a vector of 2 elements, each of them a 3 4 array). The goal is to get rid of the first two axes, but enclose will remove the last ones. So, we need to transpose the array first. 

This might be done with a dyadic transpose. However, then we would need to first find the length of the current shape, then generate the indices, rotate them and do the transpose. Instead, we will use two monadic transposes with different ranks, which will have the same effect. First, we do a transpose rotating all the elements of the shape, then we do another transpose but this one with rank ¯1, to rotate only the subarrays while leaving the first axis unchanged:

In [10]:
Assert (⍉⍤¯1⍉d) ≡ (1⌽⍳≢⍴d)⍉d     ⍝ two monadic transposes match dyadic transpose
('' '⍴' '⍴¨',¨'x') Table x (⍴x) (⍴¨x←⍉d)      ⍝ first monadic transpose (rank ⍴⍴d)
('' '⍴' '⍴¨',¨'x') Table x (⍴x) (⍴¨x←⍉⍤¯1⍉d)  ⍝ second monadic transpose (rank (⍴⍴d)-1)

┌───────────────────────┬───────┬────┐
│x │⍴x │⍴¨x │
├───────────────────────┼───────┼────┤
│┌──────┬──────┬───────┐│┌─┬─┬─┐│┌┬┬┐│
││1 │0.354 │0.702 │││2│4│3│││││││
│├──────┼──────┼───────┤│└─┴─┴─┘│├┼┼┤│
││0.57 │0.835 │0.472 ││ ││││││
│├──────┼──────┼───────┤│ │├┼┼┤│
││0.859 │¯0.224│0.203 ││ ││││││
│├──────┼──────┼───────┤│ │├┼┼┤│
││0.66 │0.364 │0.385 ││ ││││││
│└──────┴──────┴───────┘│ │└┴┴┘│
│┌──────┬──────┬───────┐│ │┌┬┬┐│
││¯0.152│¯0.272│0.805 ││ ││││││
│├──────┼──────┼───────┤│ │├┼┼┤│
││0.582 │¯0.436│3.78 ││ ││││││
│├──────┼──────┼───────┤│ │├┼┼┤│
││0.297 │1.44 │0.515 ││ ││││││
│├──────┼──────┼───────┤│ │├┼┼┤│
││1.04 │0.061 │¯0.0488││ ││││││
│└──────┴──────┴───────┘│ │└┴┴┘│
└───────────────────────┴───────┴────┘

┌──────────────────────────────┬───────┬─────┐
│x │⍴x │⍴¨x │
├──────────────────────────────┼───────┼─────┤
│┌──────┬──────┬──────┬───────┐│┌─┬─┬─┐│┌┬┬┬┐│
││1 │0.57 │0.859 │0.66 │││2│3│4││││││││
│├──────┼──────┼──────┼───────┤│└─┴─┴─┘│├┼┼┼┤│
││0.354 │0.835 │¯0.224│0.364 ││ │││││││
│├──────┼──────┼──────┼───────┤│ │├┼┼┼┤│
││0.702 │0.472 │0.203 │0.385 ││ │││││││
│└──────┴──────┴──────┴───────┘│ │└┴┴┴┘│
│┌──────┬──────┬──────┬───────┐│ │┌┬┬┬┐│
││¯0.152│0.582 │0.297 │1.04 ││ │││││││
│├──────┼──────┼──────┼───────┤│ │├┼┼┼┤│
││¯0.272│¯0.436│1.44 │0.061 ││ │││││││
│├──────┼──────┼──────┼───────┤│ │├┼┼┼┤│
││0.805 │3.78 │0.515 │¯0.0488││ │││││││
│└──────┴──────┴──────┴───────┘│ │└┴┴┴┘│
└──────────────────────────────┴───────┴─────┘

Almost there. We just need to enclose with rank two to nest the subarrays, removing the last two elements of the shape:

In [11]:
('' '⍴' '⍴¨',¨'x') Table x (⍴x) (⍴¨x←⊂⍤¯1⍉⍤¯1⍉d)

┌──────────────────────────────────────────────────────┬───┬─────────┐
│x │⍴x │⍴¨x │
├──────────────────────────────────────────────────────┼───┼─────────┤
│┌────────────────────────┬───────────────────────────┐│┌─┐│┌───┬───┐│
││1 0.57 0.859 0.66 │¯0.152 0.582 0.297 1.04 │││2│││3 4│3 4││
││0.354 0.835 ¯0.224 0.364│¯0.272 ¯0.436 1.44 0.061 ││└─┘│└───┴───┘│
││0.702 0.472 0.203 0.385│ 0.805 3.78 0.515 ¯0.0488││ │ │
│└────────────────────────┴───────────────────────────┘│ │ │
└──────────────────────────────────────────────────────┴───┴─────────┘

Done! Let's redefine our nested divide function:

In [12]:
⎕ ← ND ← ⊂∘⍉⍤¯1∘⍉⌹⍤1 2⍥(↑[0])  ⍝ nested matrix divide
('' '⍴' '⍴¨',¨'x') Table x (⍴x) (⍴¨x←b ND m)
Assert b ≡ m +.× x

┌──┴───┐
 ∘ ⍥
 ┌┴┐ ┌──┴───┐
 ⍤ ⍉ ⍤ [0]
 ┌┴┐ ┌┴┐ ┌─┘
 ∘ ¯1 ⌹ 1 2 ↑
┌┴┐
⊂ ⍉

┌──────────────────────────────────────────────────────┬───┬─────────┐
│x │⍴x │⍴¨x │
├──────────────────────────────────────────────────────┼───┼─────────┤
│┌────────────────────────┬───────────────────────────┐│┌─┐│┌───┬───┐│
││1 0.57 0.859 0.66 │¯0.152 0.582 0.297 1.04 │││2│││3 4│3 4││
││0.354 0.835 ¯0.224 0.364│¯0.272 ¯0.436 1.44 0.061 ││└─┘│└───┴───┘│
││0.702 0.472 0.203 0.385│ 0.805 3.78 0.515 ¯0.0488││ │ │
│└────────────────────────┴───────────────────────────┘│ │ │
└──────────────────────────────────────────────────────┴───┴─────────┘

Everything looks fine. To make sure our function is doing the right thing, let's try it with arrays of other ranks:

In [13]:
T ← {⍺←3 ⋄ m b←(⊂⍵)RN¨(⍺ ⍺) ⍺ ⋄ ('' '⍴' '⍴¨',¨'x') Table x (⍴x) (⍴¨x←b ND m) ⋄ Assert b≡m+.×x}  ⍝ test
T ⍬ ⋄ T 2 ⋄ T 2 2 ⋄ T 2 2 2  ⍝ scalar, vector, matrix and 3D array

┌────────────────────┬───┬────┐
│x │⍴x │⍴¨x │
├────────────────────┼───┼────┤
│┌──────┬────┬──────┐│┌─┐│┌┬┬┐│
││¯0.976│1.42│0.0886│││3│││││││
│└──────┴────┴──────┘│└─┘│└┴┴┘│
└────────────────────┴───┴────┘
┌───────────────────────────────────┬───┬───────┐
│x │⍴x │⍴¨x │
├───────────────────────────────────┼───┼───────┤
│┌───────────┬──────────┬──────────┐│┌─┐│┌─┬─┬─┐│
││¯3.67 ¯1.37│¯4.89 3.58│14.9 ¯3.53│││3│││2│2│2││
│└───────────┴──────────┴──────────┘│└─┘│└─┴─┴─┘│
└───────────────────────────────────┴───┴───────┘
┌────────────────────────────────────┬───┬─────────────┐
│x │⍴x │⍴¨x │
├────────────────────────────────────┼───┼─────────────┤
│┌──────────┬────────────┬──────────┐│┌─┐│┌───┬───┬───┐│
││0.732 1.61│¯2.25 2.25│2.46 ¯1.96│││3│││2 2│2 2│2 2││
││0.765 2.3 │¯0.973 ¯1.82│1.03 1.16││└─┘│└───┴───┴───┘│
│└──────────┴────────────┴──────────┘│ │ │
└────────────────────────────────────┴───┴─────────────┘
┌───────────────────────────────────────────┬───┬───────────────────┐
│x │⍴x │⍴¨x │
├───────────────────────────────────────────┼───┼───────────────────┤
│┌────────────┬────────────┬───────────────┐│┌─┐│┌─────┬─────┬─────┐│
││¯0.283 0.202│0.814 1.23│ 0.524 ¯0.807 │││3│││2 2 2│2 2 2│2 2 2││
││¯1.44 2 │1.48 ¯1.81│ 1.62 1.52 ││└─┘│└─────┴─────┴─────┘│
││ │ │ ││ │ │
││¯1.62 1.27 │2.56 7.03│¯0.944 ¯3.55 ││ │ │
││ 0.929 6.7 │0.0887 ¯5.61│ 0.0106 ¯0.0979││ │ │
│└────────────┴────────────┴───────────────┘│ │ │
└───────────────────────────────────────────┴───┴───────────────────┘

# Each nested

Applying a function like this, to each of the nested elements of the array but keeping the outer structure, looks like a useful operation. So, we are going to define an operator.

The `ND` function that we defined before could easily be modified to be an *each nested* operator that works with any function:

In [14]:
_EN ← {⊂⍉⍤¯1⍉⍺ ⍺⍺⍤1 2⍥(↑[0])⍵}   ⍝ each nested
 ND ← ⌹_EN
_ ← T¨⍬ 2 (2 2) (2 2 2)

However, this operator is not very general. It will work when we have matrices as right argument and vectors on the left, but it will not work for arrays of any rank. For example, let's try to use the matrix divide primitive (`⌹`) to perform a least-squares fitting with two vectors:

In [15]:
a b ← ?3 3⍴¨0                ⍝ random input data
'ab⍴⌹' Table a b (⍴a) (a⌹b)  ⍝ flat arrays just work
a b ← (⊂4 2)RN¨3 3           ⍝ random nested arrays
'ab⍴¨' Table a b (⍴a) (⍴¨a)
a ⌹_EN b

┌───────────────────┬───────────────────┬───┬──────┐
│a │b │⍴ │⌹ │
├───────────────────┼───────────────────┼───┼──────┤
│┌─────┬─────┬─────┐│┌─────┬─────┬─────┐│┌─┐│┌────┐│
││0.948│0.463│0.518│││0.115│0.414│0.535│││3│││1.23││
│└─────┴─────┴─────┘│└─────┴─────┴─────┘│└─┘│└────┘│
└───────────────────┴───────────────────┴───┴──────┘

┌───────────────────────────────────┬────────────────────────────────────────┬───┬─────────────┐
│a │b │⍴ │¨ │
├───────────────────────────────────┼────────────────────────────────────────┼───┼─────────────┤
│┌───────────┬──────────┬──────────┐│┌───────────┬─────────────┬────────────┐│┌─┐│┌───┬───┬───┐│
││0.383 1.31 │4.96 4.03 │2.31 2.76│││1.79 3.42 │0.464 0.976│2.6 1.68 │││3│││4 2│4 2│4 2││
││1.95 0.732│3.41 0.746│0.999 1.49│││3.24 0.0359│1.13 0.372│4.71 0.761││└─┘│└───┴───┴───┘│
││3.8 1.56 │3.61 4.48 │1.46 1.04│││2.46 3.39 │0.00498 0.731│0.0835 1.74 ││ │ │
││3.12 0.151│3.69 1.3 │4.89 4.49│││1.14 0.895 │3.5 1.27 │4.6 3.72 ││ │ │
│└───────────┴──────────┴──────────┘│└───────────┴─────────────┴────────────┘│ │ │
└───────────────────────────────────┴────────────────────────────────────────┴───┴─────────────┘

RANK ERROR
_EN[0] _EN←{⊂⍉⍤¯1⍉⍺ ⍺⍺⍤1 2⍥(↑[0])⍵}   ⍝ each nested
                       ∧


Instead of hardcoding the ranks inside the operator, we can check what is the rank of our nested elements looking at the first element in `⍵`. We will use this rank to transpose subarrays (`⍉⍤(⍴⍴⊃⍵)`). The rank to apply the given function is calculated from the ranks of the left and right arguments (`⊃∘⍴∘⍴¨⍺ ⍵`):

In [16]:
'⍴a' '⍴b' '⍴⍴⊃b' '⊃∘⍴∘⍴¨a b' Table (⍴a) (⍴b) (⍴⍴⊃b) (⊃∘⍴∘⍴¨a b)
_EN ← {⍺←⊢ ⋄ ⊂∘⍉⍤(⍴⍴⊃⍵)∘⍉ ⍺ ⍺⍺⍤(⊃∘⍴∘⍴¨⍺ ⍵)⍥(↑[0]) ⍵}   ⍝ each nested
 MD ← ⌹_EN
_ ← T¨⍬ 2 (2 2) (2 2 2)
'ab⌹' Table a b (a ⌹_EN b)

┌───┬───┬────┬─────────┐
│⍴a │⍴b │⍴⍴⊃b│⊃∘⍴∘⍴¨a b│
├───┼───┼────┼─────────┤
│┌─┐│┌─┐│┌─┐ │┌─┬─┐ │
││3│││3│││2│ ││1│1│ │
│└─┘│└─┘│└─┘ │└─┴─┘ │
└───┴───┴────┴─────────┘

┌───────────────────────────────────┬────────────────────────────────────────┬─────────────┐
│a │b │⌹ │
├───────────────────────────────────┼────────────────────────────────────────┼─────────────┤
│┌───────────┬──────────┬──────────┐│┌───────────┬─────────────┬────────────┐│┌───────────┐│
││0.383 1.31 │4.96 4.03 │2.31 2.76│││1.79 3.42 │0.464 0.976│2.6 1.68 │││0.882 0.845││
││1.95 0.732│3.41 0.746│0.999 1.49│││3.24 0.0359│1.13 0.372│4.71 0.761│││0.438 2 ││
││3.8 1.56 │3.61 4.48 │1.46 1.04│││2.46 3.39 │0.00498 0.731│0.0835 1.74 │││1.57 0.688││
││3.12 0.151│3.69 1.3 │4.89 4.49│││1.14 0.895 │3.5 1.27 │4.6 3.72 │││1.12 1.14 ││
│└───────────┴──────────┴──────────┘│└───────────┴─────────────┴────────────┘│└───────────┘│
└───────────────────────────────────┴────────────────────────────────────────┴─────────────┘

The `_EN` operator can now be used either to solve a linear system or perform a least-squares fitting, and it will do the right thing.

Of course, `⌹` is not the only function we can use with `_EN`. As a last example, let's take the `LF` function from the [Linear Fitting notebook](https://github.com/Dyalog/dyalog-jupyter-notebooks/blob/master/Linear%20Fitting.ipynb), which performs a linear fitting given a list of two vectors and returns the intercept, the slope and the R-squared value. For convenience, we also define a dyadic linear fitting function:

In [17]:
_R2 ← {1-(+/×⍨⍺-⍺⍺⍵)÷+/×⍨(⊢-+/÷≢)⍺}                                            ⍝ R-squared value
LF  ← {F←{((a+b∘×)_R2/⍺),⍨a b←⍵} ⋄ 0=⎕NC'⍺': ⍵F⊃⌹∘(1,⍪⍤2⍤⊢)/⍵ ⋄ ⍵F⍺(⌹/⍵-⍺ 0)}  ⍝ linear fitting
DLF ← LF⍤,⍥⊂                                                                   ⍝ dyadic linear fitting
⍝ eg with non nested values
'xy' Table x y←5×?0⍴⍨¨4 4
'abr' Table y DLF x

┌──────────────────────┬──────────────────────┐
│x │y │
├──────────────────────┼──────────────────────┤
│┌─────┬────┬────┬────┐│┌────┬────┬────┬─────┐│
││0.963│4.56│1.03│4.51│││2.51│2.79│2.63│0.762││
│└─────┴────┴────┴────┘│└────┴────┴────┴─────┘│
└──────────────────────┴──────────────────────┘

┌──────┬───────┬───────┐
│a │b │r │
├──────┼───────┼───────┤
│┌────┐│┌─────┐│┌─────┐│
││2.78│││¯0.22│││0.224││
│└────┘│└─────┘│└─────┘│
└──────┴───────┴───────┘

If we have nested data, we use the `_EN` operator:

In [18]:
'xy' Table x y←(⊂3 2)RN¨4 4
'abr' Table y DLF _EN x

┌──────────────────────────────────────────────┬─────────────────────────────────────────────┐
│x │y │
├──────────────────────────────────────────────┼─────────────────────────────────────────────┤
│┌──────────┬──────────┬────────────┬─────────┐│┌──────────┬─────────┬───────────┬──────────┐│
││2.53 3.71 │2.19 3.33 │2.92 3.7 │5 2.9 │││4.06 4.31│1.2 4.86│0.0389 4.84│1.77 4.61 ││
││4.87 0.715│2.27 4.54 │0.00102 2.32│4.22 0.99│││0.714 3.19│3.07 2 │4.03 3.15│2.81 0.144││
││3.38 1.64 │1.91 0.146│1.65 3.26│1.47 3.31│││4.03 4.31│3.46 1.04│4.19 1.19│1.35 4.68 ││
│└──────────┴──────────┴────────────┴─────────┘│└──────────┴─────────┴───────────┴──────────┘│
└──────────────────────────────────────────────┴─────────────────────────────────────────────┘

┌───────────┬────────────────┬─────────────────┐
│a │b │r │
├───────────┼────────────────┼─────────────────┤
│┌────┬────┐│┌───────┬──────┐│┌───────┬───────┐│
││2.07│5.02│││¯0.0971│¯0.108│││0.00524│0.025 ││
│├────┼────┤│├───────┼──────┤│├───────┼───────┤│
││4.21│1.97│││¯0.546 │0.0708│││0.733 │0.00749││
│├────┼────┤│├───────┼──────┤│├───────┼───────┤│
││1.65│1.84│││0.762 │0.463 │││0.256 │0.127 ││
│└────┴────┘│└───────┴──────┘│└───────┴───────┘│
└───────────┴────────────────┴─────────────────┘

It is also possible to use the `_EN` operator when both arguments do not have the same rank.

In [19]:
'xy' Table x (y1←5×?4⍴0)
'abr' Table y1 DLF _EN x

┌──────────────────────────────────────────────┬────────────────────────┐
│x │y │
├──────────────────────────────────────────────┼────────────────────────┤
│┌──────────┬──────────┬────────────┬─────────┐│┌────┬─────┬────┬──────┐│
││2.53 3.71 │2.19 3.33 │2.92 3.7 │5 2.9 │││3.44│0.785│4.93│0.0983││
││4.87 0.715│2.27 4.54 │0.00102 2.32│4.22 0.99││└────┴─────┴────┴──────┘│
││3.38 1.64 │1.91 0.146│1.65 3.26│1.47 3.31││ │
│└──────────┴──────────┴────────────┴─────────┘│ │
└──────────────────────────────────────────────┴────────────────────────┘

┌────────────┬───────────────┬──────────────┐
│a │b │r │
├────────────┼───────────────┼──────────────┤
│┌─────┬────┐│┌──────┬──────┐│┌─────┬──────┐│
││4.92 │¯16 │││¯0.824│5.37 │││0.211│0.817 ││
│├─────┼────┤│├──────┼──────┤│├─────┼──────┤│
││3.75 │2.84│││¯0.507│¯0.246│││0.242│0.0362││
│├─────┼────┤│├──────┼──────┤│├─────┼──────┤│
││0.498│1.45│││0.863 │0.411 │││0.11 │0.0753││
│└─────┴────┘│└──────┴──────┘│└─────┴──────┘│
└────────────┴───────────────┴──────────────┘

But take into account that `_EN` will deduce the shape of the elements from the first element in the right argument, so you may have to switch the arguments:

In [20]:
'xy' Table (x1←5×?4⍴0) y
'abr' Table x1 DLF⍨ _EN y

┌───────────────────┬─────────────────────────────────────────────┐
│x │y │
├───────────────────┼─────────────────────────────────────────────┤
│┌────┬─┬────┬─────┐│┌──────────┬─────────┬───────────┬──────────┐│
││4.59│4│2.75│0.784│││4.06 4.31│1.2 4.86│0.0389 4.84│1.77 4.61 ││
│└────┴─┴────┴─────┘││0.714 3.19│3.07 2 │4.03 3.15│2.81 0.144││
│ ││4.03 4.31│3.46 1.04│4.19 1.19│1.35 4.68 ││
│ │└──────────┴─────────┴───────────┴──────────┘│
└───────────────────┴─────────────────────────────────────────────┘

┌─────────────┬────────────────┬──────────────┐
│a │b │r │
├─────────────┼────────────────┼──────────────┤
│┌────┬──────┐│┌──────┬───────┐│┌─────┬──────┐│
││0.52│4.76 │││0.412 │¯0.0349│││0.168│0.0513││
│├────┼──────┤│├──────┼───────┤│├─────┼──────┤│
││3.86│0.0888│││¯0.395│0.67 │││0.227│0.624 ││
│├────┼──────┤│├──────┼───────┤│├─────┼──────┤│
││1.33│3.92 │││0.637 │¯0.367 │││0.671│0.0996││
│└────┴──────┘│└──────┴───────┘│└─────┴──────┘│
└─────────────┴────────────────┴──────────────┘

# Summary

The `_EN` operator will apply the given function to each of the nested elements of its arguments, taking as model the first element of the right argument, while keeping the outer structure.

In [21]:
]defs _EN

_EN ← {⍺←⊢ ⋄ ⊂∘⍉⍤(⍴⍴⊃⍵)∘⍉⍺ ⍺⍺⍤(⊃∘⍴∘⍴¨⍺ ⍵)⍥(↑[0])⍵}